# Feature Engineering for Portfolio Management

This notebook demonstrates the FeatureEngineeringAgent functionality:
1. Import and initialize the FeatureEngineeringAgent
2. Process sample tickers (AAPL, MSFT) to create financial features
3. Validate output and display sample data from Unity Catalog tables
4. Analyze the generated features for ML readiness

## 1. Setup and Imports

Import required libraries and initialize Spark session if needed.

In [0]:
# Import required libraries
import sys
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime

# Add the src directory to the Python path
sys.path.append('../src')

# Import our custom FeatureEngineeringAgent
from agents.feature_engineering_agent import FeatureEngineeringAgent, FeatureEngineeringError

print("✅ Libraries imported successfully")
print(f"Python path includes: {[p for p in sys.path if 'src' in p]}")

## 2. Initialize Spark Session and FeatureEngineeringAgent

Create Spark session and initialize the feature engineering agent with Unity Catalog configuration.

In [0]:
# Initialize Spark session (if not already available in Databricks)
# In Databricks, spark session is usually pre-configured
try:
    # Check if spark session already exists (common in Databricks)
    spark_session = spark
    print("✅ Using existing Spark session from Databricks")
except NameError:
    # Create new Spark session if not in Databricks environment
    spark_session = SparkSession.builder \
        .appName("FeatureEngineering") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
    print("✅ Created new Spark session")

print(f"Spark version: {spark_session.version}")
# print(f"Spark application: {spark_session.sparkContext.getConf().getAppName()}")

In [0]:
# Initialize FeatureEngineeringAgent
print("🚀 Initializing FeatureEngineeringAgent...")

# Create agent with Unity Catalog configuration
feature_agent = FeatureEngineeringAgent(catalog="portfolio_catalog", schema="portfolio_schema")

print(f"✅ FeatureEngineeringAgent initialized:")
print(f"   - Catalog: {feature_agent.catalog}")
print(f"   - Schema: {feature_agent.schema}")
print(f"   - Target namespace: {feature_agent.catalog}.{feature_agent.schema}")

## 2.5. Setup Unity Catalog Schema

Create the required catalog and schema if they don't exist. This is required for Unity Catalog environments.

In [0]:
# Setup Unity Catalog structure - Create catalog and schema if they don't exist
print("🔧 Setting up Unity Catalog structure...")

def setup_unity_catalog(catalog_name="portfolio_catalog", schema_name="portfolio_schema"):
    """Setup Unity Catalog and schema structure"""
    try:
        # Check and create catalog
        print(f"📋 Checking catalog: {catalog_name}")
        try:
            spark_session.sql(f"USE CATALOG {catalog_name}")
            print(f"✅ Catalog '{catalog_name}' exists and is accessible")
        except Exception as e:
            if "CATALOG_NOT_FOUND" in str(e):
                print(f"⚠️ Catalog '{catalog_name}' not found. Attempting to create...")
                try:
                    spark_session.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
                    print(f"✅ Created catalog: {catalog_name}")
                except Exception as create_error:
                    print(f"❌ Failed to create catalog: {str(create_error)}")
                    print("   You may need METASTORE_ADMIN permissions to create catalogs")
                    return False
            else:
                print(f"❌ Error accessing catalog: {str(e)}")
                return False
        
        # Check and create schema
        print(f"📋 Checking schema: {catalog_name}.{schema_name}")
        try:
            # Use the catalog first, then check if schema exists
            spark_session.sql(f"USE CATALOG {catalog_name}")
            
            # Check if schema exists by listing schemas
            schemas_result = spark_session.sql("SHOW SCHEMAS").collect()
            
            # Debug: Print schema structure to understand the column names
            if schemas_result:
                print(f"🔍 Debug: Schema result columns: {schemas_result[0].asDict().keys()}")
                
            # Try different possible column names for schema information
            schema_exists = False
            for row in schemas_result:
                row_dict = row.asDict()
                # Try different possible column names
                schema_value = (row_dict.get('namespace') or 
                              row_dict.get('schemaName') or 
                              row_dict.get('databaseName') or
                              row_dict.get('name') or
                              str(row_dict))
                
                print(f"🔍 Found schema entry: {schema_value}")
                if schema_value == schema_name:
                    schema_exists = True
                    break
            
            if schema_exists:
                spark_session.sql(f"USE SCHEMA {schema_name}")
                print(f"✅ Schema '{catalog_name}.{schema_name}' exists and is accessible")
            else:
                print(f"⚠️ Schema '{schema_name}' not found. Creating...")
                try:
                    spark_session.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
                    spark_session.sql(f"USE SCHEMA {schema_name}")
                    print(f"✅ Created schema: {catalog_name}.{schema_name}")
                except Exception as create_error:
                    print(f"❌ Failed to create schema: {str(create_error)}")
                    return False
                    
        except Exception as e:
            print(f"❌ Error accessing schema: {str(e)}")
            return False
        
        # Verify setup by showing current catalog and schema
        try:
            current_catalog = spark_session.sql("SELECT current_catalog()").collect()[0][0]
            current_schema = spark_session.sql("SELECT current_schema()").collect()[0][0]
            print(f"📊 Current catalog: {current_catalog}")
            print(f"📊 Current schema: {current_schema}")
            
            # Also show available schemas in the catalog for verification
            print(f"📋 Available schemas in {catalog_name}:")
            try:
                schemas_list = spark_session.sql(f"SHOW SCHEMAS IN {catalog_name}").collect()
                for schema_row in schemas_list:
                    schema_dict = schema_row.asDict()
                    # Handle different possible column names
                    schema_name_found = (schema_dict.get('namespace') or 
                                       schema_dict.get('schemaName') or 
                                       schema_dict.get('databaseName') or
                                       schema_dict.get('name') or
                                       'unknown')
                    print(f"   - {schema_name_found}")
            except Exception as show_error:
                print(f"   Could not list schemas: {str(show_error)}")
                
        except Exception as e:
            print(f"⚠️ Could not retrieve current catalog/schema info: {str(e)}")
            # Continue anyway as this is just informational
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to setup Unity Catalog: {str(e)}")
        return False

# Run the setup
setup_success = setup_unity_catalog("portfolio_catalog", "portfolio_schema")

if setup_success:
    print("\n✅ Unity Catalog setup completed successfully!")
else:
    print("\n❌ Unity Catalog setup failed. Please check permissions and configuration.")
    print("   Required permissions: USE CATALOG, CREATE SCHEMA on the catalog")
    print("   Alternative: Ask your admin to create 'portfolio_catalog.portfolio_schema' schema for you")

## 3. Check Available Raw Data

Before running feature engineering, let's verify that the raw data tables exist in Unity Catalog.

In [0]:
# Check available tables in multiple possible schemas
print("📊 Checking available raw data tables...")

# Define possible schema locations where raw data might be stored
possible_schemas = [
    "portfolio_catalog.portfolio_schema",  # Current target schema
    "finance_catalog.bronze",              # Data ingestion agent schema
    "main.finance",                        # Legacy schema
    "main.default"                         # Default schema fallback
]

raw_data_found = False
raw_data_schema = None
available_tables = []

for schema in possible_schemas:
    try:
        print(f"\n🔍 Checking schema: {schema}")
        tables = spark_session.sql(f"SHOW TABLES IN {schema}").collect()
        
        if tables:
            print(f"   📋 Found {len(tables)} tables in {schema}:")
            schema_tables = []
            for table in tables:
                table_name = table['tableName']
                schema_tables.append(table_name)
                print(f"      - {table_name}")
                
                # Check if it's a raw data table for our target tickers
                if any(ticker.lower() in table_name.lower() for ticker in ['aapl', 'msft']) or 'price' in table_name.lower():
                    raw_data_found = True
                    raw_data_schema = schema
                    available_tables.extend([f"{schema}.{table_name}"])
                    
                    # Show sample data from this table
                    print(f"        📊 Sample data from {table_name}:")
                    try:
                        sample_df = spark_session.table(f"{schema}.{table_name}")
                        
                        # Try different possible column names
                        possible_columns = ['ticker', 'symbol', 'date', 'close', 'volume']
                        available_cols = [col for col in possible_columns if col in sample_df.columns]
                        
                        if available_cols:
                            sample_df.select(*available_cols).limit(3).show()
                        else:
                            # Just show first few columns if standard ones don't exist
                            cols_to_show = sample_df.columns[:5]
                            sample_df.select(*cols_to_show).limit(3).show()
                            
                    except Exception as sample_error:
                        print(f"           ⚠️ Could not show sample: {str(sample_error)}")
        else:
            print(f"   ⚠️ No tables found in {schema}")
            
    except Exception as e:
        print(f"   ❌ Error accessing {schema}: {str(e)}")
        continue

if raw_data_found:
    print(f"\n✅ Raw data found in schema: {raw_data_schema}")
    print(f"   Available tables: {', '.join(available_tables)}")
else:
    print(f"\n⚠️ No raw data tables found in any schema!")
    print(f"   Checked schemas: {', '.join(possible_schemas)}")
    print(f"   📝 Next steps:")
    print(f"      1. Run the data ingestion notebook (01_ingest_financial_data.ipynb) first")
    print(f"      2. Or create sample data in the current schema for testing")
    
    # Offer to create sample data for testing
    print(f"\n🔧 Creating sample data for testing purposes...")
    
    # Create sample data for AAPL and MSFT
    from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, LongType
    from pyspark.sql.functions import lit
    
    sample_data = [
        ("AAPL", "2024-01-01", 150.0, 155.0, 148.0, 152.0, 151.5, 1000000),
        ("AAPL", "2024-01-02", 152.0, 158.0, 151.0, 157.0, 156.5, 1100000),
        ("AAPL", "2024-01-03", 157.0, 160.0, 155.0, 159.0, 158.5, 1200000),
        ("MSFT", "2024-01-01", 380.0, 385.0, 378.0, 383.0, 382.5, 800000),
        ("MSFT", "2024-01-02", 383.0, 388.0, 381.0, 386.0, 385.5, 850000),
        ("MSFT", "2024-01-03", 386.0, 390.0, 384.0, 388.0, 387.5, 900000)
    ]
    
    schema = StructType([
        StructField("ticker", StringType(), False),
        StructField("date", StringType(), False),
        StructField("open", DoubleType(), True),
        StructField("high", DoubleType(), True),
        StructField("low", DoubleType(), True),
        StructField("close", DoubleType(), True),
        StructField("adj_close", DoubleType(), True),
        StructField("volume", LongType(), True)
    ])
    
    try:
        sample_df = spark_session.createDataFrame(sample_data, schema)
        sample_df = sample_df.withColumn("date", sample_df.date.cast(DateType()))
        sample_df = sample_df.withColumn("ingestion_timestamp", lit("2024-01-01").cast(DateType()))
        
        # Create the sample table in portfolio_catalog.portfolio_schema
        sample_table_name = "portfolio_catalog.portfolio_schema.sample_prices"
        sample_df.write.format("delta").mode("overwrite").saveAsTable(sample_table_name)
        
        print(f"   ✅ Created sample table: {sample_table_name}")
        print(f"   📊 Sample contains {sample_df.count()} rows for AAPL and MSFT")
        
        raw_data_found = True
        raw_data_schema = "portfolio_catalog.portfolio_schema"
        available_tables = [sample_table_name]
        
    except Exception as sample_error:
        print(f"   ❌ Failed to create sample data: {str(sample_error)}")

## 4. Run Feature Engineering

Process AAPL and MSFT tickers to create financial features.

In [0]:
# Define target tickers for feature engineering
target_tickers = ["AAPL", "MSFT"]

print(f"🔧 Starting feature engineering for: {', '.join(target_tickers)}")
print(f"   Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check if we have raw data available
if not raw_data_found:
    print("❌ Cannot proceed: No raw data tables found!")
    print("   Please run the data ingestion notebook first or check the previous cell output")
    results = None
else:
    print(f"✅ Using raw data from schema: {raw_data_schema}")
    print(f"   Available tables: {', '.join(available_tables)}")
    
    try:
        # Since our existing FeatureEngineeringAgent expects a specific table structure,
        # let's create features manually using the available data
        
        results = {
            'start_time': datetime.now(),
            'processed_tickers': [],
            'failed_tickers': [],
            'total_features_created': 0
        }
        
        # Find the raw data table (could be 'prices', 'sample_prices', etc.)
        raw_table = None
        for table_path in available_tables:
            if any(name in table_path.lower() for name in ['price', 'market', 'ohlc']):
                raw_table = table_path
                break
        
        if not raw_table:
            raw_table = available_tables[0]  # Use first available table
        
        print(f"📊 Using raw data table: {raw_table}")
        
        # Read the raw data
        raw_df = spark_session.table(raw_table)
        print(f"   📈 Raw data contains {raw_df.count()} rows")
        print(f"   📋 Columns: {', '.join(raw_df.columns)}")
        
        # Process each ticker
        for ticker in target_tickers:
            try:
                print(f"\n🔧 Processing {ticker}...")
                
                # Filter data for this ticker
                ticker_df = raw_df.filter(F.col("ticker") == ticker)
                
                if ticker_df.count() == 0:
                    print(f"   ⚠️ No data found for {ticker}")
                    results['failed_tickers'].append(ticker)
                    continue
                
                # Create features using window functions
                window_spec = Window.partitionBy("ticker").orderBy("date")
                
                feature_df = ticker_df.select(
                    "ticker",
                    "date", 
                    "open",
                    "high",
                    "low",
                    "close",
                    "volume"
                ).withColumn(
                    # Daily return
                    "daily_return",
                    (F.col("close") - F.lag("close", 1).over(window_spec)) / F.lag("close", 1).over(window_spec)
                ).withColumn(
                    # 7-day moving average
                    "moving_avg_7",
                    F.avg("close").over(window_spec.rowsBetween(-6, 0))
                ).withColumn(
                    # 30-day moving average  
                    "moving_avg_30",
                    F.avg("close").over(window_spec.rowsBetween(-29, 0))
                ).withColumn(
                    # 7-day volatility (rolling standard deviation of returns)
                    "volatility_7",
                    F.stddev("daily_return").over(window_spec.rowsBetween(-6, 0))
                ).withColumn(
                    # Momentum (price change over 10 days)
                    "momentum",
                    (F.col("close") - F.lag("close", 10).over(window_spec)) / F.lag("close", 10).over(window_spec)
                ).withColumn(
                    # Feature timestamp
                    "feature_timestamp",
                    F.lit(datetime.now().date())
                )
                
                # Remove null rows (first few rows won't have complete features)
                feature_df = feature_df.filter(F.col("daily_return").isNotNull())
                
                feature_count = feature_df.count()
                print(f"   ✅ Created {feature_count} feature records for {ticker}")
                
                # Save to feature table
                feature_table_name = f"portfolio_catalog.portfolio_schema.features_{ticker}"
                
                feature_df.write \
                    .format("delta") \
                    .mode("overwrite") \
                    .saveAsTable(feature_table_name)
                
                print(f"   💾 Saved features to {feature_table_name}")
                
                results['processed_tickers'].append(ticker)
                results['total_features_created'] += feature_count
                
            except Exception as ticker_error:
                print(f"   ❌ Failed to process {ticker}: {str(ticker_error)}")
                results['failed_tickers'].append(ticker)
                continue
        
        results['end_time'] = datetime.now()
        
        if results['processed_tickers']:
            print("\n✅ Feature engineering completed!")
            print(f"   Duration: {(results['end_time'] - results['start_time']).total_seconds():.2f} seconds")
            print(f"   Total features created: {results['total_features_created']}")
            print(f"   ✅ Successfully processed: {', '.join(results['processed_tickers'])}")
            
            if results['failed_tickers']:
                print(f"   ❌ Failed tickers: {', '.join(results['failed_tickers'])}")
        else:
            print("\n❌ Feature engineering failed for all tickers")
            results = None
        
    except Exception as e:
        print(f"❌ Feature engineering failed: {str(e)}")
        print("   Check the logs for detailed error information")
        results = None

## 5. Validate Output Tables

Check that the feature tables were created successfully in Unity Catalog.

In [0]:
# Check for newly created feature tables
print("🔍 Validating output tables...")

feature_tables = []
for ticker in target_tickers:
    table_name = f"portfolio_catalog.portfolio_schema.features_{ticker}"
    
    try:
        # Check if table exists
        table_exists = spark_session.catalog.tableExists(table_name)
        
        if table_exists:
            print(f"✅ {table_name} exists")
            feature_tables.append(table_name)
            
            # Get table info
            df = spark_session.table(table_name)
            row_count = df.count()
            col_count = len(df.columns)
            
            print(f"   📊 Table stats: {row_count:,} rows, {col_count} columns")
            
            # Show column names
            print(f"   📋 Columns: {', '.join(df.columns)}")
            
        else:
            print(f"❌ {table_name} does not exist")
            
    except Exception as e:
        print(f"❌ Error checking {table_name}: {str(e)}")

print(f"\n📈 Total feature tables created: {len(feature_tables)}")

## 6. Display Sample Feature Data

Show sample data from the generated feature tables to verify quality.

In [0]:
# Display sample data from feature tables
print("📋 Sample Feature Data")
print("=" * 50)

for table_name in feature_tables:
    ticker = table_name.split('_')[-1]  # Extract ticker from table name
    
    print(f"\n🏷️ {ticker} Features ({table_name})")
    print("-" * 40)
    
    try:
        df = spark_session.table(table_name)
        
        # Show recent data (last 5 rows)
        print("\n📅 Most Recent 5 Records:")
        df.orderBy(F.desc("date")).limit(5).show(truncate=False)
        
        # Show feature summary statistics
        print("\n📊 Feature Statistics:")
        feature_cols = ['daily_return', 'moving_avg_7', 'moving_avg_30', 'volatility_7', 'momentum']
        df.select(feature_cols).summary().show()
        
    except Exception as e:
        print(f"❌ Error displaying data for {table_name}: {str(e)}")

## 7. Feature Quality Analysis

Analyze the quality and completeness of generated features.

In [0]:
# Analyze feature quality
print("🔬 Feature Quality Analysis")
print("=" * 50)

for table_name in feature_tables:
    ticker = table_name.split('_')[-1]
    
    print(f"\n📈 Analysis for {ticker}")
    print("-" * 30)
    
    try:
        df = spark_session.table(table_name)
        total_rows = df.count()
        
        # Check for null values in key features
        feature_cols = ['daily_return', 'moving_avg_7', 'moving_avg_30', 'volatility_7', 'momentum']
        
        print(f"📊 Data Completeness (out of {total_rows:,} total rows):")
        for col in feature_cols:
            null_count = df.filter(F.col(col).isNull()).count()
            non_null_count = total_rows - null_count
            completeness = (non_null_count / total_rows) * 100 if total_rows > 0 else 0
            
            status = "✅" if completeness >= 95 else "⚠️" if completeness >= 80 else "❌"
            print(f"   {status} {col}: {completeness:.1f}% complete ({non_null_count:,} values)")
        
        # Check date range
        date_stats = df.select(
            F.min("date").alias("min_date"),
            F.max("date").alias("max_date"),
            F.count("date").alias("total_days")
        ).collect()[0]
        
        print(f"\n📅 Date Range:")
        print(f"   From: {date_stats['min_date']}")
        print(f"   To: {date_stats['max_date']}")
        print(f"   Total trading days: {date_stats['total_days']:,}")
        
        # Feature value ranges
        print(f"\n📏 Feature Ranges:")
        for col in ['daily_return', 'volatility_7', 'momentum']:
            stats = df.select(
                F.min(col).alias('min_val'),
                F.max(col).alias('max_val'),
                F.avg(col).alias('avg_val')
            ).collect()[0]
            
            print(f"   {col}: [{stats['min_val']:.4f}, {stats['max_val']:.4f}] (avg: {stats['avg_val']:.4f})")
        
    except Exception as e:
        print(f"❌ Error analyzing {table_name}: {str(e)}")

## 8. Verification Queries

Run some verification queries to ensure data consistency and feature correctness.

In [0]:
# Run verification queries
print("🔍 Data Verification Queries")
print("=" * 50)

if feature_tables:
    # Query 1: Check if daily returns are calculated correctly
    print("\n📊 Query 1: Daily Return Calculation Verification")
    print("Checking if daily_return = (close - prev_close) / prev_close")
    
    for table_name in feature_tables[:1]:  # Check first table only
        ticker = table_name.split('_')[-1]
        
        verification_query = f"""
        SELECT 
            ticker,
            date,
            close,
            LAG(close, 1) OVER (PARTITION BY ticker ORDER BY date) as prev_close,
            daily_return,
            ROUND(
                (close - LAG(close, 1) OVER (PARTITION BY ticker ORDER BY date)) / 
                LAG(close, 1) OVER (PARTITION BY ticker ORDER BY date), 
                6
            ) as calculated_return
        FROM {table_name}
        WHERE date >= (SELECT MAX(date) - INTERVAL 7 DAYS FROM {table_name})
        ORDER BY date DESC
        LIMIT 5
        """
        
        try:
            result = spark_session.sql(verification_query)
            print(f"\n{ticker} - Recent daily returns:")
            result.show(truncate=False)
        except Exception as e:
            print(f"❌ Error in verification query: {str(e)}")

    # Query 2: Check moving averages
    print("\n📈 Query 2: Moving Average Verification")
    print("Checking 7-day and 30-day moving averages")
    
    for table_name in feature_tables[:1]:  # Check first table only
        ticker = table_name.split('_')[-1]
        
        ma_query = f"""
        SELECT 
            date,
            close,
            moving_avg_7,
            moving_avg_30,
            ROUND(
                AVG(close) OVER (
                    PARTITION BY ticker 
                    ORDER BY date 
                    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
                ), 2
            ) as calculated_ma7
        FROM {table_name}
        WHERE date >= (SELECT MAX(date) - INTERVAL 10 DAYS FROM {table_name})
        ORDER BY date DESC
        LIMIT 5
        """
        
        try:
            result = spark_session.sql(ma_query)
            print(f"\n{ticker} - Recent moving averages:")
            result.show(truncate=False)
        except Exception as e:
            print(f"❌ Error in moving average query: {str(e)}")

else:
    print("⚠️ No feature tables available for verification")

## 9. Summary and Next Steps

Summarize the feature engineering results and provide guidance for next steps.

In [0]:
# Final summary
print("📋 Feature Engineering Summary")
print("=" * 50)

if 'results' in locals() and results:
    print(f"\n✅ Feature Engineering Completed Successfully")
    print(f"   - Target tickers: {', '.join(target_tickers)}")
    print(f"   - Successfully processed: {len(results['processed_tickers'])} tickers")
    print(f"   - Failed: {len(results['failed_tickers'])} tickers")
    print(f"   - Total features created: {results['total_features_created']:,}")
    print(f"   - Processing time: {(results['end_time'] - results['start_time']).total_seconds():.2f} seconds")
    
    if results['processed_tickers']:
        print(f"\n📊 Available Feature Tables:")
        for ticker in results['processed_tickers']:
            print(f"   - portfolio_catalog.portfolio_schema.features_{ticker}")
            
    print(f"\n🎯 Created Features:")
    feature_list = [
        "daily_return (Daily price return)",
        "moving_avg_7 (7-day moving average)",
        "moving_avg_30 (30-day moving average)",
        "volatility_7 (7-day rolling volatility)",
        "momentum (Price momentum indicator)",
        "feature_timestamp (Feature creation date)"
    ]
    
    for feature in feature_list:
        print(f"   ✅ {feature}")
        
else:
    print("❌ Feature engineering did not complete successfully")
    print("   Please check the error messages above and retry")

print(f"\n🚀 Next Steps:")
print(f"   1. Review the generated features for data quality")
print(f"   2. Use these feature tables for ML model training")
print(f"   3. Set up scheduled jobs for regular feature updates")
print(f"   4. Consider adding more advanced features (technical indicators, etc.)")
print(f"   5. Implement feature monitoring and alerting")

print(f"\n✨ Feature engineering notebook completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")